# コード5-2：拡散方程式の数値計算の実装

『ウイルス感染の数理モデルとシミュレーション ーデータを定量的に理解するー』

拡散方程式の数値計算の実装の例をまとめる。

## ライブラリ、パッケージ（モジュール）の読み込み

In [2]:
## 必要があればインストール
# install.packages("ggplot2", "patchwork", "deSolve)

In [ ]:
rm(list = ls(all = TRUE))
library(ggplot2)
library(patchwork)
library(deSolve)

## 拡散方程式の数値計算


数学コラム０４：ウイルス感染の時空間動態

二次元平面$(x,y) \in \mathbb{R}^2$の領域$\Omega \in \mathbb{R}^2$における、時刻$0<t<T$での個体数$u(t,x,y)$（数学コラム０４では$n(t,x,y)$であるが、法線ベクトル$n$特別するため$u$を用いた）に対する拡散方程式

$$
\frac{\partial u(t,x,y)}{\partial t} = D\left( \frac{\partial^2 u(t,x,y)}{{\partial x}^2} + \frac{\partial^2 u(t,x,y)}{{\partial y}^2} \right) = D\Delta u(t,x,y)
$$

について、数値計算を行う。

Variational formulationは以下になる。

$$
\int_{\Omega}{\left\{ u^{m+1}v + \tau D \nabla u^{m+1} \cdot \nabla v \right\} } - \int_{\partial \Omega}{\tau D \frac{\partial u^{m+1}}{\partial n}}v - \int_{\Omega}{ u^{m}v } = 0
$$

ここで、$u^m$は$u(m \tau,x,y)$であり、$v$はテスト関数、$\tau$は微小時間（時間方向の数値計算のタイムステップ）、$n$は境界$\partial \Omega$で定義された外向きの法線ベクトルを表す（時間方向の離散化やFreeFEMで用いられる弱形式の導出については、FreeFEMのドキュメンテーションページ https://doc.freefem.org/models/evolution-problems.html や他の解説などを参照）。また、

$$
\frac{\partial u^{m+1}}{\partial n} = \nabla u^{m+1} \cdot n
$$

である。

斉次Dirichlet境界条件

$$
u(t,x,y) = 0 \quad \text{on } \partial \Omega \times ]0,T[
$$

と、Neumann境界条件のうちゼロフラックス境界条件（斉次Neumann境界条件）

$$
\frac{\partial u \left( t, x, y \right)}{\partial n} = 0 \quad \text{on } \partial \Omega \times ]0,T[
$$

を考える。

FreeFEM++とPythonからFreeFEMの実行可能ファイル（.edp）の生成とその実行ができるpyfreefemパッケージを用いる。

- FreeFEM - An open-source PDE Solver using the Finite Element Method https://freefem.org/
- pyfreefem · PyPI https://pypi.org/project/pyfreefem/